In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import gc
from keras import backend as K
from keras.layers import Dense,Input,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import re
from unidecode import unidecode
from sklearn.preprocessing import StandardScaler
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt # for plotting
# Any results you write to the current directory are saved as output.

In [2]:
EMBEDDING_FILE = '../input/fasttext-russian-2m/wiki.ru.vec'

train = pd.read_csv('../input/avito-demand-prediction/train.csv', usecols=['description', 'deal_probability'])
test = pd.read_csv('../input/avito-demand-prediction/test.csv', usecols=['description'])

In [7]:
special_character_removal = re.compile(r'[^A-Za-z\.\-\?\!\,\#\@\% ]',re.IGNORECASE)
def clean_text(x):
    x_ascii = unidecode(x)
    x_clean = special_character_removal.sub('',x_ascii)
    return x_clean

train['description'] = train['description'].apply(lambda x: clean_text(str(x)))
test['description'] = test['description'].apply(lambda x: clean_text(str(x)))

In [42]:
train['description'] = train['description'].astype(str)
test["description"] = test['description'].astype(str)
train["description"].fillna("что нибудь")
test["description"].fillna("что нибудь")

cols = "description"
train[cols + '_num_words'] = train[cols].apply(lambda comment: len(comment.split())) # Count number of Words
train[cols + '_num_unique_words'] = train[cols].apply(lambda comment: len(set(w for w in comment.split())))
train[cols + '_words_vs_unique'] = train[cols+'_num_unique_words'] / train[cols+'_num_words'] * 100
train[cols + '_num_chars'] = train[cols].apply(len)

test[cols + '_num_words'] = test[cols].apply(lambda comment: len(comment.split())) # Count number of Words
test[cols + '_num_unique_words'] = test[cols].apply(lambda comment: len(set(w for w in comment.split())))
test[cols + '_words_vs_unique'] = test[cols+'_num_unique_words'] / test[cols+'_num_words'] * 100
test[cols + '_num_chars'] = test[cols].apply(len)


In [43]:
features = train[['description_num_chars', 'description_words_vs_unique','description_num_unique_words']]
test_features = test[['description_num_chars', 'description_words_vs_unique','description_num_unique_words']]
features = features.fillna(0)
test_features = test_features.fillna(0)
ss = StandardScaler()
ss.fit(np.vstack([features, test_features]))
features = ss.transform(features)
test_features = ss.transform(test_features)

In [44]:
X_train = train["description"]
y_train = train[["deal_probability"]].values

del train
gc.collect()

X_test = test["description"]
del test
gc.collect()

Max_features - Max number of words used in the tokenizer, the more features the better I asssume

Maxlen - The cutoff point of word length in a description

Embed_size - Since we are using pretrained fasttext embeddings, it is already trained for 300 dimensions

In [45]:
max_features=20000
maxlen=100
embed_size=300

In [46]:
from keras.preprocessing import text, sequence
tok=text.Tokenizer(num_words=max_features)
tok.fit_on_texts(X_train)
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
del X_train
gc.collect()
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
del X_test
gc.collect()

In [47]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float16')
        embeddings_index[word] = coefs


In [48]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [49]:
del tok
gc.collect()



Please change the size of GRU and CONV1D to 128 and 64 preferably, used a smaller size to get quick outputs

In [51]:
#def root_mean_squared_error(y_true, y_pred):
#    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 
features_input = Input(shape=(features.shape[1],))
sequence_input = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
x = SpatialDropout1D(0.5)(x)
#x = Bidirectional(LSTM(32, return_sequences=True,dropout=0.1,recurrent_dropout=0.1, return_state = True))(x)
x, x_h, x_c = Bidirectional(GRU(32, return_sequences=True,dropout=0.1,recurrent_dropout=0.1, return_state = True))(x)
#x = Bidirectional(GRU(32, return_sequences=True,dropout=0.1,recurrent_dropout=0.1, return_state = True))(x)
#x = Conv1D(16, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)

x = concatenate([avg_pool, max_pool, x_h, features_input]) 
preds = Dense(1, activation="sigmoid")(x)
model = Model([sequence_input, features_input], preds)
model.compile(loss='MSE',optimizer=Adam(lr=1e-3),metrics=['accuracy',root_mean_squared_error])

In [52]:
length_train = len(x_train)
X_tra = x_train[int(length_train/10):]
features_tra = features[int(length_train/10):]
X_val = x_train[:int(length_train/10)]
features_val = features[:int(length_train/10)]
y_tra = y_train[int(length_train/10):]
y_val = y_train[:int(length_train/10)]

In [53]:
# filepath="../input/best-model/best.hdf5"
batch_size = 1000
epochs = 2
filepath="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
callbacks_list = [checkpoint, early]

In [54]:
del x_train
gc.collect()
del y_train
gc.collect()

In [ ]:
model.fit([X_tra, features_tra], y_tra, batch_size=batch_size, epochs=epochs, validation_data=([X_val, features_val], y_val),callbacks = callbacks_list,verbose=1)
#Loading model weights
model.load_weights(filepath)
print('Predicting....')
y_pred = model.predict([x_test, test_features],batch_size=1024,verbose=1)

In [ ]:
sub = pd.read_csv('../input/avito-demand-prediction/sample_submission.csv')
sub['deal_probability'] = y_pred
sub['deal_probability'].clip(0.0, 1.0, inplace=True)
sub.to_csv('gru_cnn_description.csv', index=False)